In [25]:
import pandas as pd
import numpy
import re
import spacy

# Loading the datasets:  combined publications, category definitions, area definitions, individual publications

In [26]:
df1 = pd.read_csv("C:/Users/harsh/Desktop/project/data/Combined_Names_Descriptions_Titles.csv", header=0,
                    delimiter=",",encoding = "ISO-8859-1")                             # dataframe of the combined publications                              




df2 = pd.read_csv("C:/Users/harsh/Desktop/project/data/CEC_definitions_final.csv", header=0,
                    delimiter=",",encoding = "ISO-8859-1")                                                     # dataframe of the categories                   

df2 = df2.drop_duplicates()
df2  =df2.reset_index()
df2= df2.reset_index()
df2.rename(columns = {'level_0':'ID'}, inplace = True)
df2 = df2[['ID','Category','def']]


df3 = pd.read_csv("C:/Users/harsh/Desktop/project/data/CEC_area_definitions.csv", header=0,
                    delimiter=",",encoding = "ISO-8859-1")                                     # dataframe of the areas           


df3 = df3.reset_index()
df3.rename(columns = {'index':'ID'}, inplace = True)


df4 = pd.read_csv("C:/Users/harsh/Desktop/project/data/Final_Data_For_NLP.csv", header=0,
                    delimiter=",",encoding = "ISO-8859-1")                                              
df4 = df4[['ID','Name','descriptions']]                                           # dataframe of the individual publications


# View first 5 rows of dataframes

In [27]:
df1[:5]

,Unnamed: 0,Name,Description_Title
0,0,abolfazl safikhani,abolfazl safikhani currently assistant profes...
1,1,aditya johri,aditya johri studies use information communic...
2,2,ahmed bin zaman,ever tell story let say friends called zoo ba...
3,3,alexander brodsky,alex brodsky joined george mason university 1...
4,4,alexander levis,alexander h levis university professor electr...


In [28]:
df2[559:564]

,ID,Category,def
559,559,interdependencies,interdependencies the dependence of two or mor...
560,560,value chain,value chain is a progression of activities tha...
561,561,digital value chain,digital value chain is a set of processes that...
562,562,native american,native american a member of any of the indigen...
563,563,infrastructure,infrastructure is the set of facilities and sy...


In [29]:
df3[:5]

,ID,area,def
0,0,artificial intelligence,contrived art rather nature artificially forma...
1,1,autonomous systems,of political bodies existing independent entit...
2,2,computer engineering,machine performing calculations automatically ...
3,3,computer networks communications,machine performing calculations automatically ...
4,4,cybersecurity,na


In [30]:
df4[:5]

,ID,Name,descriptions
0,0,abolfazl safikhani,abolfazl safikhani currently assistant profess...
1,1,aditya johri,aditya johri studies use information communica...
2,2,ahmed bin zaman,ever tell story let say friends called zoo bad...
3,3,alexander brodsky,alex brodsky joined george mason university 19...
4,4,alexander levis,alexander h levis university professor electri...


In [31]:
nlp = spacy.load('en_core_web_sm')      #loading the english dictionary from spacy package 

# Defining a helper function for stemming and lemmatizing the text 

In [32]:
def stem(text):                                                                        
	x = []
	doc = nlp(str(text))

	for word in doc:
		x.append(word.lemma_)
	return " ".join(x)

In [33]:
df1['Description_Title']= df1['Description_Title'].apply(stem)          # applying helper function to the dataframes

df2['def']= df2['def'].apply(stem)

df3['def']= df3['def'].apply(stem)

df4['descriptions'] = df4['descriptions'].apply(stem)

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()                                                 # initializing the count vectorizer 

# Vectorizing using combined publications dataframe

In [35]:
fit = cv.fit(df1['Description_Title'])                                                  


author_vectors=fit.transform(df1['Description_Title']).toarray()

category_vectors = fit.transform(df2['def']).toarray()

area_Vectors = fit.transform(df3['def']).toarray()

# Vectorizing using individual publications dataframe


In [36]:
pub_fit = cv.fit(df4['descriptions'])                                                         
pub_author_vectors=pub_fit.transform(df4['descriptions']).toarray()

pub_category_vectors = pub_fit.transform(df2['def']).toarray()

pub_area_Vectors = pub_fit.transform(df3['def']).toarray()


# Helper function to calculate the cosine similarity

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_sim_vectors(vec1, vec2):                                                   
    vec1 = vec1.reshape(1,-1)
    vec2 = vec2.reshape(1,-1)

    return cosine_similarity(vec1,vec2)[0][0]

# Looping through the combined authors vectors and calculating cosine similarity with each category and creating a list of scores

In [38]:
cosine_sim_list_cat = []                    
 
for i in range(len(author_vectors)):
    dummy = []
    for j in range(len(category_vectors)):
        similarity = cosine_sim_vectors(author_vectors[i],category_vectors[j])
        dummy.append(similarity)
    cosine_sim_list_cat.append(dummy)
    

# Looping through the combined authors vectors and calculating cosine similarity with each area and creating a list of scores

In [39]:
cosine_sim_list_area = []

for i in range(len(author_vectors)):                                        
    dummy = []
    for j in range(len(area_Vectors)):
        similarity = cosine_sim_vectors(author_vectors[i],area_Vectors[j])
        dummy.append(similarity)
    cosine_sim_list_area.append(dummy)
    

# Looping through the pub_authors vectors and calculating cosine similarity with each category and creating a list of scores

In [40]:
pub_cosine_sim_list_cat = []

for i in range(len(pub_author_vectors)):                       
    dummy = []
    for j in range(len(pub_category_vectors)):
        similarity = cosine_sim_vectors(pub_author_vectors[i],pub_category_vectors[j])
        dummy.append(similarity)
    pub_cosine_sim_list_cat.append(dummy)
    

# Looping through the pub_authors vectors and calculating cosine similarity with each area and creating a list of scores

In [41]:
pub_cosine_sim_list_area = []

for i in range(len(pub_author_vectors)):
    dummy = []
    for j in range(len(pub_area_Vectors)):
        similarity = cosine_sim_vectors(pub_author_vectors[i],pub_area_Vectors[j])
        dummy.append(similarity)
    pub_cosine_sim_list_area.append(dummy)
    


# Looping through the category vectors and calculating cosine similarity with each author and creating a list of scores

In [57]:
person_cosine_sim_list = []

for i in range(len(category_vectors )):                       
    dummy = []
    for j in range(len(author_vectors)):
        similarity = cosine_sim_vectors(category_vectors [i],author_vectors[j])
        dummy.append(similarity)
    person_cosine_sim_list.append(dummy)


# Looping through the category vectors and calculating cosine similarity with each author and creating a list of scores

In [83]:
area_person_cosine_sim_list = []

for i in range(len(area_Vectors)):                       
    dummy = []
    for j in range(len(author_vectors)):
        similarity = cosine_sim_vectors(area_Vectors [i],author_vectors[j])
        dummy.append(similarity)
    area_person_cosine_sim_list.append(dummy)

# Looping through the cosine sim list and taking the top 10 category names and scores

In [58]:
top10_cat = []                                   
score_cat = []
for i in range(0,len(cosine_sim_list_cat)):                             
    top10 = sorted(list(enumerate(cosine_sim_list_cat[i])),reverse = True, key = lambda x:x[1])[0:10]
    dummy = []
    score_dummy = []
    for j in top10:
        a = j[0]
        b = j[1]
        category_name = df2['Category'][a]
        dummy.append(category_name)
        score_dummy.append(b)
    top10_cat.append(dummy)
    score_cat.append(score_dummy)

# Looping through the cosine sim list and taking the top 10 area names and scores

In [59]:
top10_area = []            
score_area = []
for i in range(0,len(cosine_sim_list_area)):                                  
    top10 = sorted(list(enumerate(cosine_sim_list_area[i])),reverse = True, key = lambda x:x[1])[0:10]
    dummy = []
    score_dummy = []
    for j in top10:
        a = j[0]
        b = j[1]
        area_name = df3['area'][a]
        dummy.append(area_name)
        score_dummy.append(b)
    top10_area.append(dummy)
    score_area.append(score_dummy)

# Looping through the cosine sim list and taking the top 10 category names for each individual publication

In [60]:
pub_top10_cat = []
pub_score_cat = []
for i in range(0,len(pub_cosine_sim_list_cat)): 
    top10 = sorted(list(enumerate(pub_cosine_sim_list_cat[i])),reverse = True, key = lambda x:x[1])[0:10]
    dummy = []
    score_dummy = []
    for j in top10:
        a = j[0]
        b = j[1]
        category_name = df2['Category'][a]
        dummy.append(category_name)
        score_dummy.append(b)
    pub_top10_cat.append(dummy)
    pub_score_cat.append(score_dummy)

# Looping through the cosine sim list and taking the top 10 area names for each individual publication

In [61]:
pub_top10_area = []
pub_score_area = []
for i in range(0,len(pub_cosine_sim_list_area)):
    top10 = sorted(list(enumerate(pub_cosine_sim_list_area[i])),reverse = True, key = lambda x:x[1])[0:10]
    dummy = []
    score_dummy = []
    for j in top10:
        a = j[0]
        b = j[1]
        area_name = df3['area'][a]
        dummy.append(area_name)
        score_dummy.append(b)
    pub_top10_area.append(dummy)
    pub_score_area.append(score_dummy)

# Looping through the cosine sim list and taking the top 10 author names for each category

In [62]:
pub_top10_person = []
pub_score_person = []
for i in range(0,len(person_cosine_sim_list)):
    top10 = sorted(list(enumerate(person_cosine_sim_list[i])),reverse = True, key = lambda x:x[1])[0:10]
    dummy = []
    score_dummy = []
    for j in top10:
        a = j[0]
        b = j[1]
        name = df1['Name'][a]
        dummy.append(name)
        score_dummy.append(b)
    pub_top10_person.append(dummy)
    pub_score_person.append(score_dummy)

# Looping through the cosine sim list and taking the top 10 author names for each area

In [84]:
top10_person_area = []
score_person_area = []
for i in range(0,len(area_person_cosine_sim_list)):
    top10 = sorted(list(enumerate(area_person_cosine_sim_list[i])),reverse = True, key = lambda x:x[1])[0:10]
    dummy = []
    score_dummy = []
    for j in top10:
        a = j[0]
        b = j[1]
        name = df1['Name'][a]
        dummy.append(name)
        score_dummy.append(b)
    top10_person_area.append(dummy)
    score_person_area.append(score_dummy)

In [85]:
df1['similar_categories'] = top10_cat                 #adding the list to the dataframe
df1['category_score'] = score_cat
df1['similar_area'] = top10_area
df1['area_score'] = score_area

df4['similar_categories'] = pub_top10_cat
df4['category_score'] = pub_score_cat
df4['similar_area'] = pub_top10_area
df4['area_score'] = pub_score_area

df2['top10'] = pub_top10_person
df2['top10_scores']= pub_score_person

df3['top10'] = top10_person_area
df3['top10_scores']= score_person_area

In [86]:
cat_df= df1[['Name','Description_Title','similar_categories','category_score']]       #creating  new dataframes with the required columns
area_df = df1[['Name','Description_Title','similar_area','area_score']]
pub_cat_df= df4[['Name','descriptions','similar_categories','category_score']]
pub_area_df = df4[['Name','descriptions','similar_area','area_score']]

top10_authors_cat = df2[['Category','top10','top10_scores']]
top10_authors_area = df3[['area','top10','top10_scores']]

# View first 5 rows of the output dataframes

In [87]:
cat_df[:5]

,Name,Description_Title,similar_categories,category_score
0,abolfazl safikhani,abolfazl safikhani currently assistant profe...,"[stochastic modeling, modeling nanoparticles, ...","[0.3258514946996161, 0.3232521973429334, 0.323..."
1,aditya johri,aditya johri study use information communica...,"[results clinical studies, bioengineering, eng...","[0.2769955015366262, 0.27643225241166486, 0.27..."
2,ahmed bin zaman,ever tell story let say friend call zoo bad ...,"[structural engineering, structural stress, st...","[0.3581828950543877, 0.33878704316095354, 0.32..."
3,alexander brodsky,alex brodsky join george mason university 19...,"[model based systems engineering, network mode...","[0.2560685111848679, 0.22484437003969773, 0.22..."
4,alexander levis,alexander h levis university professor elect...,"[network modeling, model based systems enginee...","[0.31516382127767395, 0.313822808174314, 0.310..."


In [88]:
area_df[:5]

,Name,Description_Title,similar_area,area_score
0,abolfazl safikhani,abolfazl safikhani currently assistant profe...,"[stochastic modeling, statistical modeling, ap...","[0.3258514946996161, 0.3192429620458017, 0.129..."
1,aditya johri,aditya johri study use information communica...,"[design results clinical studies, visual compu...","[0.27032910538576066, 0.21385780181719366, 0.1..."
2,ahmed bin zaman,ever tell story let say friend call zoo bad ...,"[structural engineering, autonomous systems, e...","[0.3581828950543877, 0.09065956796066224, 0.09..."
3,alexander brodsky,alex brodsky join george mason university 19...,"[stochastic modeling, statistical modeling, ap...","[0.18918082523955362, 0.18388119261399738, 0.1..."
4,alexander levis,alexander h levis university professor elect...,"[stochastic modeling, statistical modeling, bi...","[0.27222136005626746, 0.26661380395023954, 0.1..."


In [89]:
pub_cat_df[:5]

,Name,descriptions,similar_categories,category_score
0,abolfazl safikhani,abolfazl safikhani currently assistant profess...,"[statistical modeling, stochastic modeling, pr...","[0.16037507477489607, 0.14433756729740646, 0.1..."
1,aditya johri,aditya johri study use information communicati...,"[bioengineering, engineering computing student...","[0.15474611514754325, 0.13257401008712377, 0.1..."
2,ahmed bin zaman,ever tell story let say friend call zoo bad re...,"[neural interfaces, architecture, hardware, ai...","[0.158113883008419, 0.1224744871391589, 0.1217..."
3,alexander brodsky,alex brodsky join george mason university 1993...,"[subsystems, neural networks, neuroscience, co...","[0.18519709381740776, 0.136147096098501, 0.113..."
4,alexander levis,alexander h levis university professor electri...,"[subsystems, bioengineering, neuroscience, dat...","[0.1858288839247533, 0.153301007957441, 0.1517..."


In [90]:
pub_area_df[:5]

,Name,descriptions,similar_area,area_score
0,abolfazl safikhani,abolfazl safikhani currently assistant profess...,"[statistical modeling, stochastic modeling, ed...","[0.16037507477489607, 0.14433756729740646, 0.1..."
1,aditya johri,aditya johri study use information communicati...,"[data analytics, education, design results cli...","[0.12930163676704717, 0.11788145340170698, 0.1..."
2,ahmed bin zaman,ever tell story let say friend call zoo bad re...,"[computational biomedicine, software engineeri...","[0.10170952554312154, 0.0939336436627724, 0.07..."
3,alexander brodsky,alex brodsky join george mason university 1993...,"[data analytics, energy, computer networks com...","[0.08548569901134032, 0.08423853269968526, 0.0..."
4,alexander levis,alexander h levis university professor electri...,"[software engineering, visual computing, compu...","[0.1040849940787939, 0.0870034280745056, 0.085..."


In [92]:
top10_authors_cat[:5]

,Category,top10,top10_scores
0,cognitive systems,"[janis terpenny, jim chen, amarda shehu, ray h...","[0.1708341315141826, 0.14954998827887028, 0.13..."
1,intelligent agents,"[gheorghe tecuci, mihai boicu, jim chen, sanma...","[0.11724547106430705, 0.11078950953330423, 0.1..."
2,knowledge representation reasoning,"[mihai boicu, gheorghe tecuci, jim chen, samue...","[0.141288203564421, 0.13392502573750803, 0.108..."
3,machine learning,"[gheorghe tecuci, mihai boicu, aditya johri, r...","[0.26394415454644365, 0.22742026590093653, 0.1..."
4,natural language processing,"[duminda wijesekera, rajesh ganesan, daniel ba...","[0.20385771488056054, 0.1986324846786106, 0.19..."


In [93]:
top10_authors_area[:5]

,area,top10,top10_scores
0,artificial intelligence,"[myeong lee, sadegh torabi, diana wang, Ã¶zlem...","[0.2908402250838777, 0.17516394906194158, 0.16..."
1,autonomous systems,"[janis terpenny, jim chen, amarda shehu, thoma...","[0.15794679858946128, 0.14186342071370853, 0.1..."
2,computer engineering,"[gheorghe tecuci, mihai boicu, janis terpenny,...","[0.2383102303944741, 0.21215305220381353, 0.18..."
3,computer networks communications,"[myeong lee, ali k raz, daniel barbara, dumind...","[0.2680245533862051, 0.23733426053465734, 0.19..."
4,cybersecurity,"[martin slawski, kuo tian, celso moller ferrei...","[0.05397405462930527, 0.03193110314562001, 0.0..."


# Converting dataframes to csv

In [94]:
cat_df.to_csv("C:/Users/harsh/Desktop/project/output/cosine_similarity_category.csv")
area_df.to_csv("C:/Users/harsh/Desktop/project/output/cosine_similarity_area.csv")
pub_cat_df.to_csv("C:/Users/harsh/Desktop/project/output/pub_cosine_similarity_category.csv")
pub_area_df.to_csv("C:/Users/harsh/Desktop/project/output/pub_cosine_similarity_area.csv")
top10_authors_cat.to_csv("C:/Users/harsh/Desktop/project/output/top10_authors_cat.csv")
top10_authors_area.to_csv("C:/Users/harsh/Desktop/project/output/top10_authors_area.csv")